# Importing Libraries & Dependencies

In [1]:
import math
import random

# Defining User Parameters & Constants

In [2]:
pop_size = 100
mutation_rate = 0.1
tournament_size = 2
selection_rate = 0.5
num_generations = 100

# Search space limits
x_min, x_max = -65.536, 65.536
y_min, y_max = -65.536, 65.536

In [3]:
# Shekel's foxholes constants

a = [[-32, -16, 0, 16, 32] for i in range(25)]

c = [0.5, 0.2, 0.2, 0.4, 0.6, 0.3, 0.7, 0.5, 0.3, 0.7,     
     0.5, 0.2, 0.2, 0.4, 0.6, 0.3, 0.7, 0.5, 0.3, 0.7,     
     0.5, 0.2, 0.2, 0.4, 0.6]

# Shakel's Function

In [4]:
def shekels_foxholes(x, y):
    result = 0
    for i in range(25):
        numerator = (x - a[i][0])**2 + (y - a[i][1])**2
        denominator = c[i] + numerator
        result -= 1 / denominator
    return result


# Initialize the population

In [5]:
def initialize_population(pop_size):
    population = []
    for i in range(pop_size):
        x = random.uniform(x_min, x_max)
        y = random.uniform(y_min, y_max)
        individual = (x, y)
        population.append(individual)
    return population

# Evaluate the fitness of each individual in the population

In [6]:
def evaluate_fitness(population):
    fitness = []
    for individual in population:
        x, y = individual
        fitness.append(shekels_foxholes(x, y))
    return fitness

# Perform proportional selection to choose parents

In [7]:
def proportional_selection(population, p):
    fitness = evaluate_fitness(population)
    total_fitness = sum(fitness)
    probabilities = [f/total_fitness for f in fitness]
    parents = []
    for i in range(len(population)):
        candidates = random.choices(population, weights=probabilities, k=p)
        best_candidate = min(candidates, key=lambda individual: shekels_foxholes(*individual))
        parents.append(best_candidate)
    return parents

# Perform truncation selection to choose parents

In [8]:
def truncation_selection(population, p):
    k = int(len(population) * p)  # Keep top p% of population
    fitness = []
    for individual in population:
        if isinstance(individual, tuple):
            x, y = individual
        else:
            x, y = (individual,)
        fitness.append(shekels_foxholes(x, y))
    sorted_population = [x for _, x in sorted(zip(fitness, population), key=lambda pair: pair[0])]
    selected_individuals = sorted_population[-k:]
    while len(selected_individuals) < len(population):
        selected_individuals.append(random.choice(selected_individuals))
    return selected_individuals

# Perform deterministic tournament selection to choose parents

In [9]:
def deterministic_tournament_selection(population, tournament_size):
    parents = []
    while len(parents) < len(population):
        candidates = random.sample(population, tournament_size)
        best_candidate = min(candidates, key=lambda individual: shekels_foxholes(*individual))
        parents.append(best_candidate)
    return parents

# Perform linear ranking selection to choose parents

In [10]:
def linear_ranking_selection(population, min_p, max_p):
    fitness = evaluate_fitness(population)
    sorted_population = [x for _, x in sorted(zip(fitness, population))]
    parents = []
    N = len(population)
    rank_prob = [1/N * (min_p + (max_p - min_p) * (i - 1)/(N - 1)) for i in range(1, N + 1)]
    for i in range(N):
        rand_num = random.uniform(0, 1)
        for j in range(N):
            if rand_num <= sum(rank_prob[:j+1]):
                parents.append(sorted_population[j])
                break
    return parents

# Perform Stochastic Binary Tournament selection to choose parents

In [11]:
def stochastic_binary_tournament(population, selection_prob):
    
    # Calculate fitness of population
    fitness = evaluate_fitness(population)
    
    # Select parents using binary tournament
    parents = []
    while len(parents) < len(population):
        idx1 = stochastic_binary_tournament_helper(fitness, selection_prob)
        idx2 = stochastic_binary_tournament_helper(fitness, selection_prob)
        parents.append(population[idx1])
        parents.append(population[idx2])
    
    # Return the selected parents
    return parents


def stochastic_binary_tournament_helper(fitness, selection_prob):
    # Select two individuals at random
    idx1 = random.randint(0, len(fitness) - 1)
    idx2 = random.randint(0, len(fitness) - 1)
    
    # Determine which individual is better
    if fitness[idx1] > fitness[idx2]:
        better_idx = idx1
        worse_idx = idx2
    else:
        better_idx = idx2
        worse_idx = idx1
    
    # Determine the probability of selecting the better individual
    p_better = selection_prob if random.random() > 0.5 else 1 - selection_prob
    
    # Select the better or worse individual with the appropriate probability
    if random.random() < p_better:
        return better_idx
    else:
        return worse_idx

# One point crossover

In [12]:
def one_point_crossover(parent1, parent2):
    # Choose a random crossover point
    crossover_point = random.randint(1, len(parent1) - 1)
    # Create the two offspring by combining parent1 and parent2
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
    # Return the offspring
    return offspring1, offspring2

# Mutation

In [13]:
def mutation(individual, mutation_rate):
    # Generate a random number for each gene in the individual
    mutated_genes = [gene + random.uniform(-1, 1) * mutation_rate for gene in individual]
    # Return the mutated individual
    return tuple(mutated_genes)

# Main Function

In [14]:
def main(population, selection_method, crossover_method, mutation_method, mutation_rate, num_generations):
    
    for i in range(num_generations):
        
        # Select parents using any of the selection technique
        
        if selection_method == "proportional":
            parents = proportional_selection(population, p)
            
        elif selection_method == "truncation":
            parents = truncation_selection(population, p)
            
        elif selection_method == "deterministic_tournament":
            parents = deterministic_tournament_selection(population, tournament_size)
            
        elif selection_method == "linear_ranking":
            parents = linear_ranking_selection(population, min_p, max_p) 
            
        elif selection_method == "stochastic_binary_tournament":
            parents = stochastic_binary_tournament(population, p)
            
        else:
            raise ValueError("Invalid selection method specified")

        # Create offspring
        
        offspring = []
        for i in range(int(pop_size/2)):
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            offspring1, offspring2 = crossover_method(parent1, parent2)
            offspring.append(offspring1)
            offspring.append(offspring2)

        # Mutate offspring
        
        for i in range(len(offspring)):
            if random.uniform(0, 1) < mutation_rate:
                offspring[i] = mutation_method(offspring[i], mutation_rate)

        # Evaluate offspring fitness
        offspring_fitness = evaluate_fitness(offspring)

        # Replace population with offspring
        population = offspring

    # Return best individual found in the final generation
    best_individual = max(population, key=lambda individual: shekels_foxholes(*individual))
    return best_individual

# Program Execution

In [15]:
while True:
    
    selection_mechanism = input('\nChoose a selection mechanism (proportional, truncation, deterministic_tournament, linear_ranking, stochastic_binary_tournament), or type "exit" to quit: ')
    
    if selection_mechanism == "exit":
        break
    
    elif selection_mechanism not in ["proportional", "truncation", "deterministic_tournament", "linear_ranking", "stochastic_binary_tournament"]:
        print("Invalid selection mechanism. Please choose a valid selection mechanism or type 'exit' to quit.")
        continue

    if selection_mechanism == "proportional":
        
        # Proportional selection
        proportional_params = [1, 2, 5]
        best_individuals = []

        for p in proportional_params:
            selection_method = lambda population: proportional_selection(population, p)
            best_individual = main(initialize_population(pop_size), selection_mechanism, one_point_crossover, mutation, mutation_rate, num_generations)
            best_individuals.append((best_individual, shekels_foxholes(*best_individual)))

        best_individuals = sorted(best_individuals, key=lambda x: x[1], reverse=True)
        best_individual = best_individuals[0][0]
        
        print(f"\n\nPROPORTIONAL SELECTION \n\nParams for proportional selection are : {proportional_params[0]}, {proportional_params[1]}, and {proportional_params[2]} ")

        print(f"Best individual found: {best_individual} \nFitness: {shekels_foxholes(*best_individual)}")
        
    elif selection_mechanism == "truncation":
        
        # Truncation selection
        truncation_params = [0.1, 0.3, 0.5]
        best_individuals = []

        for p in truncation_params:
            selection_mechanism = lambda population: truncation_selection(population, p)
            best_individual = main(initialize_population(pop_size), "truncation", one_point_crossover, mutation, mutation_rate, num_generations)
            best_individuals.append((best_individual, shekels_foxholes(*best_individual)))

        best_individuals = sorted(best_individuals, key=lambda x: x[1], reverse=True)
        best_individual = best_individuals[0][0]
        
        print(f"\n\nTRUNCATION SELECTION \n\nParams for truncation selection are : {truncation_params[0]}, {truncation_params[1]}, and {truncation_params[2]} ")
        
        print(f"Best individual found: {best_individual}, \nFitness: {shekels_foxholes(*best_individual)}")
        
    elif selection_mechanism == "linear_ranking":
        
        linear_ranking_params = [(0.8, 1.2), (0.9, 1.1), (1.0, 1.5)]
        best_individuals = []

        for min_p, max_p in linear_ranking_params:
            selection_method = lambda population: linear_ranking_selection(population, min_p, max_p)
            best_individual = main(initialize_population(pop_size), selection_mechanism, one_point_crossover, mutation, mutation_rate, num_generations)
            best_individuals.append((best_individual, shekels_foxholes(*best_individual)))
        
        best_individuals = sorted(best_individuals, key=lambda x: x[1], reverse=True)
        best_individual = best_individuals[0][0]

        print(f"\n\nLINEAR RANKING SELECTION \n\nParams (min, max) for linear ranking selection are : {linear_ranking_params[0]}, {linear_ranking_params[1]}, and {linear_ranking_params[2]} ")

        print(f"Best individual found: {best_individual},\nFitness: {shekels_foxholes(*best_individual)}")

    elif selection_mechanism == "deterministic_tournament":
        deterministic_tournament_params = [2, 5, 10]
        for p in deterministic_tournament_params:
            selection_method = lambda population: deterministic_tournament_selection(population, tournament_size)
            best_individual = main(initialize_population(pop_size), selection_mechanism, one_point_crossover, mutation, mutation_rate, num_generations)
            best_individuals.append((best_individual, shekels_foxholes(*best_individual)))

        best_individuals = sorted(best_individuals, key=lambda x: x[1], reverse=True)
        best_individual = best_individuals[0][0]

        print(f"\n\nDETERMINISTIC TOURNAMENT SELECTION \n\nParams for deterministic tournament selection are : {deterministic_tournament_params[0]}, {deterministic_tournament_params[1]}, and {deterministic_tournament_params[2]} ")

        print(f"Best individual found: {best_individual}, \nFitness: {shekels_foxholes(*best_individual)}")

    elif selection_mechanism == "stochastic_binary_tournament":

        selection_probabilities = [0.8, 0.6, 0.4]

        for p in selection_probabilities:
            selection_method = lambda population: stochastic_binary_tournament(population, p)
            best_individual = main(initialize_population(pop_size), selection_mechanism, one_point_crossover, mutation, mutation_rate, num_generations)
            best_individuals.append((best_individual, shekels_foxholes(*best_individual)))

        best_individuals = sorted(best_individuals, key=lambda x: x[1], reverse=True)
        best_individual = best_individuals[0][0]

        print(f"\n\nSTOCHASTIC BINARY TOURNAMENT SELECTION\n\nSelection probabilities used are: {selection_probabilities[0]}, {selection_probabilities[1]}, and {selection_probabilities[2]} ")

        print(f"Best individual found: {best_individual}, \nFitness: {shekels_foxholes(*best_individual)}")



Choose a selection mechanism (proportional, truncation, deterministic_tournament, linear_ranking, stochastic_binary_tournament), or type "exit" to quit: proportional


PROPORTIONAL SELECTION 

Params for proportional selection are : 1, 2, and 5 
Best individual found: (-29.204773979797388, -18.130659069985693) 
Fitness: -1.9564070251748935

Choose a selection mechanism (proportional, truncation, deterministic_tournament, linear_ranking, stochastic_binary_tournament), or type "exit" to quit: truncation


TRUNCATION SELECTION 

Params for truncation selection are : 0.1, 0.3, and 0.5 
Best individual found: (68.17707067943802, 67.44924483656023), 
Fitness: -0.0014706185168998718

Choose a selection mechanism (proportional, truncation, deterministic_tournament, linear_ranking, stochastic_binary_tournament), or type "exit" to quit: deterministic_tournament


DETERMINISTIC TOURNAMENT SELECTION 

Params for deterministic tournament selection are : 2, 5, and 10 
Best individual found: (68.177